In [11]:
%reset
import parcels
from parcels import ParticleFile, GeographicPolar
import numpy as np
import pandas as pd
import xarray as xr
from matplotlib import pyplot as plt
import datetime as dati
from operator import attrgetter
from math import sqrt, pi, cos
import warnings as wr

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


## Coords are the same for all months

In [12]:
# For working on home computer
path = ".."

scen = np.array(["S0", "S1", "S2", "S3", "S4", "S5", "S6"])
mon = np.array(["may", "jun", "jul", "aug", "sep", "oct"])

cs = xr.open_dataset(path + "\\data\\coordinates.nc").load()
lon_grid = cs.lon_grid.values
lat_grid = cs.lat_grid.values
dep_grid = cs.dep_grid.values
cs.close()

npart = 8 # will be squared, then doubled

# Releases 30 particles at grid of lats, lons and deps every ten hours
lon_range = np.linspace(lon_grid[4,4], lon_grid[37,38], num = npart)
lat_range = np.linspace(lat_grid[4,4], lat_grid[37,38], num = npart)

# Create a grid of particles
plons = np.repeat(lon_range, npart)
plons = np.repeat(plons, 2)
plats = np.tile(lat_range, npart)
plats = np.repeat(plats, 2)
pdeps = np.repeat(4.8, repeats = npart*npart)
pdeps = np.concatenate([pdeps, np.repeat(21.5, repeats = npart*npart)], axis = 0)

def DeleteParticle(particle, fieldset, time): 
            particle.delete() # Deletes particle if it's out of bounds

#del cs, lon_range, lat_range

In [13]:
# Check they're unique!!
print(np.shape(plats), np.shape(plons), np.shape(pdeps))

x = np.empty(shape = (3,128))
for i in range(128): x[:,i] = [plats[i], plons[i], pdeps[i]]

done = np.unique(x, axis = 1)
print(np.shape(done))

#fig = plt.figure()
#ax = plt.axes()
#donplot = plt.scatter(done[1], done[0], c = done[2])
#print(done[:,0:5])
done2 = np.concatenate([done[2,4:36],done[2,4:36]])

#fig2 = plt.figure()
#ax2 = plt.axes()
#donplot2 = plt.scatter(done[1], done[0], c = done2)

plats = done[0]
plons = done[1]
pdeps = done2
print(np.shape(plats), np.shape(plons), np.shape(pdeps))

(128,) (128,) (128,)
(3, 64)
(64,) (64,) (64,)


# All in one go!

In [14]:
scen = np.array(["S3" ])
mons = np.array(["oct"])

for mo in range(1):
    zarrname = "D:OceanParcels\\outputs2\\" + scen[mo] + "_" + mons[mo] + ".zarr"
    dfile = "D:OceanParcels\\data\\final grids\\final_" + mons[mo] + "_" + scen[mo] + "_1.nc"
    print("Starting ", zarrname)

    ds = xr.open_dataset(dfile).load()
    times_secs = ds.times_secs.values
    U_atten = ds.U_atten.values
    V_atten = ds.V_atten.values
    W = ds.W.values
    ds.close()

    # Define fieldset
    vals = {'U': U_atten, 'V': V_atten, 'W': W}

    dims = {'U': {'lon': lon_grid, 'lat': lat_grid, 'depth': dep_grid,  'time': times_secs},
            'V': {'lon': lon_grid, 'lat': lat_grid, 'depth': dep_grid,  'time': times_secs},
            'W': {'lon': lon_grid, 'lat': lat_grid, 'depth': dep_grid,  'time': times_secs}}

    fset = parcels.FieldSet.from_data(data = vals, dimensions = dims,
                      chunksize = 'auto', # allow_time_extrapolation = True,
                      gridindexingtype = 'nemo')

    del U_atten, V_atten, W, ds

    pset = parcels.ParticleSet.from_list(fieldset = fset, pclass = parcels.ScipyParticle, 
                                 lon = plons, lat = plats, depth = pdeps, 
                                 time = np.ones(npart*npart)*fset.gridset.dimrange('time')[0],
                                 repeatdt = dati.timedelta(hours = 14).total_seconds())
    pkern = pset.Kernel(parcels.AdvectionRK4_3D)
    pset.populate_indices()

    outfile = pset.ParticleFile(name = zarrname, outputdt = dati.timedelta(hours = 1))

    pset.execute(pkern, output_file = outfile, dt = dati.timedelta(minutes = 30),
                 recovery = {parcels.ErrorCode.ErrorOutOfBounds: DeleteParticle})

    del pset, fset, pkern

Starting  D:OceanParcels\outputs2\S3_oct.zarr
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
INFO: Output files are stored in D:OceanParcels\outputs2\S3_oct.zarr.
  0%|          | 0/2677800.0 [00:00<?, ?it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
  0%|          | 10800.0/2677800.0 [00:23<1:35:13, 466.81it/s]Correct cell not found after 100000 iterations
  1%|          | 14400.0/2677800.0 [00:31<1:36:04, 462.02it/s]Correct cell not found after 100000 iterations
  1%|          | 14400.0/2677800.0 [00:41<1:36:04, 462.02it/s]Correct cell not found after 100000 iterat

  9%|▉         | 237600.0/2677800.0 [10:12<2:25:58, 278.59it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
  9%|▉         | 244800.0/2677800.0 [10:26<1:47:04, 378.70it/s]Correct cell not found after 100000 iterations
  9%|▉         | 248400.0/2677800.0 [10:34<1:41:28, 399.05it/s]Correct cell not found after 100000 iterations
 10%|▉         | 259200.0/2677800.0 [10:48<1:03:38, 633.34it/s]Correct cell not found after 100000 iterations
 10%|▉         | 259200.0/2677800.0 [11:02<1:03:38, 633.34it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 10%|▉         | 262800.0/2677800.0 [11:10<1:58:32, 339.56it/s]Correct cell not found after 100000 iterations
 10%|▉         | 262800.0/2677800.0 [11:22<1:58:32, 339.56it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 10%|▉         | 266400.0/2677800.0 [11:32<2:35:24, 258.62it/s]Correct cell not found aft

Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 19%|█▉        | 522000.0/2677800.0 [22:39<5:23:00, 111.23it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 19%|█▉        | 522000.0/2677800.0 [22:52<5:23:00, 111.23it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after

 24%|██▍       | 648000.0/2677800.0 [32:25<3:00:06, 187.83it/s]Correct cell not found after 100000 iterations
 24%|██▍       | 651600.0/2677800.0 [32:32<2:27:08, 229.52it/s]Correct cell not found after 100000 iterations
 24%|██▍       | 655200.0/2677800.0 [32:40<2:04:33, 270.62it/s]Correct cell not found after 100000 iterations
 25%|██▍       | 658800.0/2677800.0 [32:47<1:47:52, 311.95it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 25%|██▍       | 658800.0/2677800.0 [33:03<1:47:52, 311.95it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 25%|██▍       | 662400.0/2677800.0 [33:19<2:45:11, 203.34it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 25%|██▌       | 669600.0/2677800.0 [33:33<1:48:15, 309.18it/s]Correct cell not found after 100000 iterat

Correct cell not found after 100000 iterations
 30%|███       | 813600.0/2677800.0 [42:54<1:17:32, 400.65it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 31%|███       | 817200.0/2677800.0 [43:01<1:51:26, 278.25it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 31%|███       | 820800.0/2677800.0 [43:14<1:51:41, 277.10it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 31%|███       | 820800.0/2677800.0 [43:34<1:51:41, 277.10it/s]Correct cell not found after 100000 iterations
 31%|███       | 824400.0/2677800.0 [43:35<2:12:13, 233.61it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 31%|███       | 831600.0/2677800.0 [43:50<1:32:23, 333.05it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 31%|███   

...

 76%|███████▌  | 2041200.0/2677800.0 [1:53:48<41:25, 256.12it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 76%|███████▋  | 2044800.0/2677800.0 [1:53:58<49:26, 213.41it/s]Correct cell not found after 100000 iterations
 76%|███████▋  | 2048400.0/2677800.0 [1:54:05<39:59, 262.33it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 77%|███████▋  | 2052000.0/2677800.0 [1:54:18<39:30, 263.97it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 77%|███████▋  | 2055600.0/2677800.0 [1:54:37<43:30, 238.39it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 77%|███████▋  | 2055600.0/2677800.0 [1:54:48<43:30, 238.39it/s]Corre

 81%|████████  | 2156400.0/2677800.0 [2:03:11<30:50, 281.69it/s]Correct cell not found after 100000 iterations
 81%|████████  | 2167200.0/2677800.0 [2:03:20<14:41, 579.53it/s]Correct cell not found after 100000 iterations
 82%|████████▏ | 2185200.0/2677800.0 [2:03:35<06:55, 1186.84it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 82%|████████▏ | 2188800.0/2677800.0 [2:03:44<11:10, 729.69it/s] Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 82%|████████▏ | 2196000.0/2677800.0 [2:03:55<10:30, 764.14it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 82%|████████▏ | 2203200.0/2677800.0 [2:04:06<10:33, 748.80it/s]Correct cell not found after 100000 iterations
 82%|████████▏ | 2206800.0/2677800.0 [2:04:11<10:49, 724.71it/s]Correct cell not found after 100000 iterations
 83%|████████▎ | 2210400.0/2677800.0 [2:04:16<11:01, 706.24it/s]Correct cell not

 88%|████████▊ | 2365200.0/2677800.0 [2:11:39<22:35, 230.65it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 88%|████████▊ | 2368800.0/2677800.0 [2:11:45<22:59, 224.04it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 89%|████████▊ | 2372400.0/2677800.0 [2:11:53<19:15, 264.36it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 89%|████████▊ | 2376000.0/2677800.0 [2:12:05<18:24, 273.28it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 89%|████████▊ | 2376000.0/2677800.0 [2:12:19<18:24, 273.28it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 89%|████████▉ | 2379600.0/2677800.0 [2:12:26<21:22, 232.54it/s]Correct cell not found after 100000 iterations
Corre

 92%|█████████▏| 2476800.0/2677800.0 [2:19:20<18:06, 184.99it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 93%|█████████▎| 2480400.0/2677800.0 [2:19:42<21:48, 150.85it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 93%|█████████▎| 2480400.0/2677800.0 [2:20:00<21:48, 150.85it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 93%|█████████▎| 2484000.0/2677800.0 [2:20:09<22:25, 144.03it/s]Correct cell not found after 100000 ite

 98%|█████████▊| 2628000.0/2677800.0 [2:27:30<03:58, 208.62it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 98%|█████████▊| 2631600.0/2677800.0 [2:27:43<04:04, 188.63it/s]Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
Correct cell not found after 100000 iterations
 98%|█████████▊| 2635200.0/2677800.0 [2:28:00<03:40, 193.63it/s]Correct cell not found after 100000 iterations
 99%|█████████▊| 2638800.0/2677800.0 [2:28:06<02:38, 245.55it/s]Correct cell not found after 100000 iterations
 99%|█████████▊| 2642400.0/2677800.0 [2:28:11<01:57, 302.22it/s]Correct cell not found after 100000 iterations
 99%|█████████▉| 2646000.0/2677800.0 [2:28:17<01:28, 359.26it/s]Correct cell not found after 100000 iterations
 99%|

In [15]:
#%%script echo Skipped!

import sys, glob

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

[('x', 3200),
 ('ParticleFile', 1480),
 ('GeographicPolar', 1072),
 ('attrgetter', 904),
 ('done2', 624),
 ('pdeps', 624),
 ('dims', 232),
 ('vals', 232),
 ('dfile', 198),
 ('mon', 184),
 ('zarrname', 156),
 ('DeleteParticle', 144),
 ('dep_grid', 144),
 ('open', 144),
 ('done', 128),
 ('lat_grid', 128),
 ('lon_grid', 128),
 ('mons', 124),
 ('scen', 120),
 ('cs', 112),
 ('lat_range', 112),
 ('lon_range', 112),
 ('plats', 112),
 ('plons', 112),
 ('times_secs', 112),
 ('cos', 72),
 ('dati', 72),
 ('np', 72),
 ('pd', 72),
 ('plt', 72),
 ('sqrt', 72),
 ('wr', 72),
 ('xr', 72),
 ('path', 51),
 ('outfile', 48),
 ('i', 28),
 ('npart', 28),
 ('mo', 24),
 ('pi', 24)]